<a href="https://colab.research.google.com/github/itsvipa/ctm-codebar-nov-22-planning/blob/main/Codebar_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install thefuzz[speedup]
!pip install py-stringsimjoin

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 175 kB 6.2 MB/s 
     |████████████████████████████████| 2.2 MB 36.8 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.1 MB 5.1 MB/s 
     |████████████████████████████████| 661 kB 46.2 MB/s 
  Created wheel for py-stringsimjoin: filename=py_stringsimjoin-0.3.2-cp37-cp37m-linux_x86_64.whl size=3724130 sha256=b57aa3b9412fe53e5af0d10d29273a3ec0fa2583bc8ec090636dedde6ea3140e
  Stored in directory: /root/.cache/pip/wheels/7c/0d/a4/c0a779a4d196bed8ed41ca12be09a93e158e4fc974b59f9fd5
  Created wheel for py-stringmatching: filename=py_stringmatching-0.4.2-cp37-cp37m-linux_x86_64.whl size=2055525 sha256=a25ef1e63be670115039f48838fbb12e46ddf52f53ccd197b5cd9ebb82773c9b
  Stored in directory: /root/.cache/pip/wheels/7e/12/86/46f9378ae730550dd2f89cad50880e70d66cabedd

In [ ]:
import requests
from bs4 import BeautifulSoup
import re
import json
import csv
import pandas
import random
import math
from os.path import exists
from concurrent.futures import ThreadPoolExecutor
from thefuzz import fuzz, process
import py_stringsimjoin as ssj
import py_stringmatching as sm

In [ ]:
example_url = 'https://priceable.co.uk/product/32734'

In [ ]:
def post_with_retry(url, payload, attempt=0, max_retries=3):
  result = requests.post(url, json=payload)
  if result.status_code == 503 and attempt < max_retries:
    return post_with_retry(url, payload, attempt + 1, max_retries)
  return result


def get_product_from_api(id):
  payload = {'id': "{}".format(id), 'retailer': "all"}
  url = "https://priceable.co.uk/api/product/"
  product_result = post_with_retry(url, payload)
  product = {}
  if product_result.status_code == 200:
    product =  product_result.json()['product']
  product['status_code'] = product_result.status_code
  return product

def get_related_products_from_api(id, name=''):
  payload = {'id': "{}".format(id), 'retailer': "all", 'name': name}
  url = "https://priceable.co.uk/api/related-products"
  return requests.post(url, json=payload).json()


def get_product(id, save_related=False):
  if id % 100 == 0:
    print('getting product ', id)
  product = get_product_from_api(id)
  if product['status_code'] == 200 and save_related:
    related = get_related_products_from_api(id, name=product['name'])
    product['related'] = [r['id'] for r in related]
  return product

def get_product_price_rows(product, product_field_names, price_field_names):
  product['price_count'] = len(product['pricing'])
  product_row = {k:product[k] for k in product_field_names}
  
  if product_row.get('skus'):
    product_row['skus'] = list(product_row['skus'].values())
  
  prices = [{**product_row, **p} for p in product['pricing']]
  price_rows = [{k:p[k] for k in price_field_names} for p in prices]

  return product_row, price_rows

In [ ]:
product_field_names = [
    'id',
    'name',
    'measure_quantity',
    'measure_amount',
    'measure_unit',
#   'related',
    'skus',
    'price_count']

price_field_names = [
    'id',
    'name',
    'retailer',
    'available',
    'price',
    'price_per_unit',
    'price_unit_amount',
    'price_unit',
    'crawl_date']

def retrieve_statuses(start=0, end=10, status_file="status.csv.gz"):
  if exists(status_file):
    status_list = pandas.read_csv(status_file)['status'].values.tolist()
    current_status_size = len(status_list)
    
    if end > current_status_size:
      for i in range(current_status_size - 1, end):
        status_list.append(0)
    
    return status_list
  
  status_list = [0 for i in range(end)]
  return status_list


def write_statuses(status_list, status_file):
  df = pandas.DataFrame(status_list, columns=['status'])
  df.to_csv(status_file)

def retrieve_product_batch(ids):
  with ThreadPoolExecutor(max_workers=3) as executor:
    results = executor.map(get_product, ids)
  
  return { i:r for i, r in zip(ids, results) }

def retrieve_products(
    start=0,
    end=10,
    status_file="/content/drive/MyDrive/codebar_data/status.csv",
    product_file="/content/drive/MyDrive/codebar_data/products.csv",
    related_file="/content/drive/MyDrive/codebar_data/related.csv",
    price_file="/content/drive/MyDrive/codebar_data/prices.csv",
    batch_size=1000):

  status_list = retrieve_statuses(start, end, status_file)

  price_file_exists = exists(price_file)
  product_file_exists = exists(product_file)

  num_batches = math.ceil((end - start) / batch_size)

  with open(product_file, 'a', newline='') as product_csv, open(price_file, 'a', newline='') as price_csv:
    
    product_writer = csv.DictWriter(product_csv, fieldnames=product_field_names)
    price_writer = csv.DictWriter(price_csv, fieldnames=price_field_names)
    
    if not product_file_exists:
      product_writer.writeheader()
    
    if not price_file_exists:
      price_writer.writeheader()
    
    for batch_n in range(num_batches):
      print('executing batch: ', batch_n)
      batch_start = start + batch_n * batch_size
      batch_end = min(batch_start + batch_size, end)
      ids = [i for i in range(batch_start, batch_end) if status_list[i] not in (200, 404)]
      product_batch = retrieve_product_batch(ids)
      print('finished retrieving batch')
      price_rows = []
      product_rows = []
      product_price_rows = [
          get_product_price_rows(p, product_field_names, price_field_names)
          for p in product_batch.values() if p['status_code'] == 200]
      
      for product, prices in product_price_rows:
        product_rows.append(product)
        price_rows = price_rows + prices

      print('batch product count: ', len(product_rows))

      if len(product_rows) > 0:
        product_writer.writerows(product_rows)
        print('updated product file')

        price_writer.writerows(price_rows)
        print('updated price file')

      for k, v in product_batch.items():
        status_list[k] = v['status_code']
      write_statuses(status_list, status_file)
      print('updated status')


In [ ]:
# retrieve_products(20000, 150000)

In [ ]:
df = pandas.read_csv('/content/drive/MyDrive/codebar_data/products.csv')
df.sort_values(by=['price_count'], ascending=False).head(100)

,id,name,measure_quantity,measure_amount,measure_unit,skus,price_count
8338,28798,Tetley Original Tea Bags x240,240.0,750,g,"['256093024', '3124932', '113676011', '32744', '48468011', '5000208069897', 'b40a4ef3-fcfb-454d-...",8
12849,33714,Hellmann's Real Squeezy Mayonnaise,NaN,430,ml,"['7010738', '111631011', '910000020309', '264365273', '13959011', 'c9bda77f-9187-4a1a-b8f8-48751...",8
8348,28808,Sharp's Doom Bar Amber Ale,NaN,500,ml,"['260342577', '6691085', '38703011', '122199011', '910001057649', '662439b2-7eba-4731-9efc-2414f...",8
7710,28107,St Austell Proper Job IPA,NaN,500,ml,"['264792757', '7707305', '55542011', '910001696166', '5028403155146', '15452ba9-1fd5-4b8e-913f-a...",8
8375,28836,PG tips Original Tea Bags 240 pack,240.0,696,g,"['287195415', '2827018', '280635011', '910001958290', '13240011', '8712100712494', 'f41c1606-17a...",8
...,...,...,...,...,...,...,...
11538,32243,Border Dark Chocolate Gingers,NaN,150,g,"['6327553', '481759011', '483739011', '1000134959173', '304416699', '6dce3854-bf2c-41f7-aa65-980...",7
14032,35024,Sensodyne Sensitivity & Gum Original Sensitive Toothpaste,NaN,75,ml,"['1000108538926', '303014342', '7954218', '451224011', '445892011', '80221f52-57bc-42b9-ad14-178...",7
7924,28346,Schar Gluten Free Wholesome White Loaf,NaN,300,g,"['7791746', '320902011', '300389011', '910002521596', '290819809', '905dd6f4-05ea-4a32-8e45-d14d...",7
6670,26992,Twinings Herbal Peppermint Tea Bags 20s,20.0,40,g,"['254198819', '20186', '6617766', '39876011', '113653011', 'eca4a1e7-0541-45d1-9f0d-27ded19aa9eb...",7


To Do:
- Get Related product info
- Write something to download all the products
- Figure out a method to match products with nutritional api

- Trim down the csvs

In [ ]:
nutrition = pandas.read_csv('/content/drive/MyDrive/codebar_data/ukproducts.csv')
products = pandas.read_csv('/content/drive/MyDrive/codebar_data/products.csv')

In [ ]:
ssj.profile_table_for_join(products)

,Unique values,Missing values,Comments
Attribute,,,
id,81670 (100.0%),0 (0.0%),This attribute can be used as a key attribute.
name,78814 (96.5%),0 (0.0%),
measure_quantity,132 (0.16%),68345 (83.68%),Joining on this attribute will ignore 68345 (83.68%) rows.
measure_amount,970 (1.19%),23791 (29.13%),Joining on this attribute will ignore 23791 (29.13%) rows.
measure_unit,93 (0.11%),24276 (29.72%),Joining on this attribute will ignore 24276 (29.72%) rows.
skus,81638 (99.96%),0 (0.0%),
price_count,9 (0.01%),0 (0.0%),


In [ ]:
ssj.profile_table_for_join(nutrition)

,Unique values,Missing values,Comments
Attribute,,,
_id,87049 (100.0%),0 (0.0%),This attribute can be used as a key attribute.
categories_properties_tags,1196 (1.37%),680 (0.78%),Joining on this attribute will ignore 680 (0.78%) rows.
nutrient_levels,1378 (1.58%),1089 (1.25%),Joining on this attribute will ignore 1089 (1.25%) rows.
nutrient_levels_tags,179 (0.21%),1089 (1.25%),Joining on this attribute will ignore 1089 (1.25%) rows.
quantity,2967 (3.41%),67958 (78.07%),Joining on this attribute will ignore 67958 (78.07%) rows.
nutriments,64755 (74.39%),680 (0.78%),Joining on this attribute will ignore 680 (0.78%) rows.
product_name,64821 (74.46%),5122 (5.88%),Joining on this attribute will ignore 5122 (5.88%) rows.
ecoscore_data,85515 (98.24%),905 (1.04%),Joining on this attribute will ignore 905 (1.04%) rows.
stores_tags,891 (1.02%),58766 (67.51%),Joining on this attribute will ignore 58766 (67.51%) rows.


In [ ]:
alnum_tok = sm.AlphanumericTokenizer(return_set=True)

In [ ]:
nutrition['new_key_attr'] = range(0, len(nutrition))

output_pairs = ssj.jaccard_join(
    products, nutrition, 
    'id', 'new_key_attr', 'name', 'product_name', 
    alnum_tok, 0.3, 
    l_out_attrs=['name'], r_out_attrs=['product_name'])

0% [############################# ] 100% | ETA: 00:00:00

In [ ]:
deduped_pairs = output_pairs.sort_values('_sim_score', ascending=False).drop_duplicates(['l_id'])
deduped_pairs

,_id,l_id,r_new_key_attr,l_name,r_product_name,_sim_score
30616,30616,27071,34361,Sprite,Sprite,1.0
22318,22318,38962,25683,Coca-Cola,Coca Cola,1.0
22778,22778,25386,26154,IRN-BRU,IRN BRU,1.0
4018,4018,26121,4179,Lucozade Zero Pink Lemonade,Zero Lucozade Pink Lemonade,1.0
22689,22689,94806,26012,Willie's Cacao Sea Salt Caramel Pearls,Willie's Cacao Sea Salt Caramel Pearls,1.0
...,...,...,...,...,...,...
36114,36114,86230,39124,L'Oreal Elvive Colour Protect Conditioner,Elvive Colour Protect,0.5
36115,36115,99794,39124,L'Oreal Elvive Colour Protect Shampoo,Elvive Colour Protect,0.5
36116,36116,49749,39125,Feather & Down Massage To Sleep,Feather & Down Sleep Butter,0.5
36118,36118,52854,39125,Feather & Down Sleep Well Set,Feather & Down Sleep Butter,0.5
